In [1]:
import sys
sys.path.append('../')
sys.path.append('../old/')

import matplotlib.pyplot as plt

import numpy as np
from scipy import rand
from scipy.constants import e
import geom_impact_poly as poly
import FiniteDifferences_ShortleyWeller_SquareGrid as PIC_FDSW
import FFT_OpenBoundary_SquareGrid as PIC_FFT
import FFT_PEC_Boundary_SquareGrid as PIC_PEC_FFT

from pypic import PyPIC_Fortran_M2P_P2M, PyPIC
from meshing import RectMesh2D
from poisson_solver import FD_solver as FD
from poisson_solver import FFT_solver as FFT
from poisson_solver.FD_solver import laplacian_2D_5stencil

PyKLU not found


In [2]:
na = np.array
Dh =1e-1
N_part_gen = 100000

tree = [[0,0],
		[1.,0],
		[1., 1,],
		[5.,1.],
		[2.,4.],
		[4,4],
		[2,7],
		[3,7],
		[1,9],
		[2,9],
		[0,11]]
		
tree=np.array(tree)
x_tree = tree[:,0]
y_tree = tree[:,1]

y_tree -= 6.

x_aper = 6.
y_aper = 7.

x_tree = np.array([0.]+ list(x_tree)+[0.])
y_tree = np.array([-y_aper]+ list(y_tree)+[y_aper])


		


x_part = x_aper*(2.*rand(N_part_gen)-1.)
y_part = y_aper*(2.*rand(N_part_gen)-1.)

x_on_tree = np.interp(y_part, y_tree, x_tree)

mask_keep = np.logical_and(np.abs(x_part)<x_on_tree, np.abs(x_part)>x_on_tree*0.8)
x_part = x_part[mask_keep]
y_part = y_part[mask_keep]

nel_part = 0*x_part+1.


		


chamber = poly.polyg_cham_geom_object({'Vx':na([x_aper, -x_aper, -x_aper, x_aper]),
									   'Vy':na([y_aper, y_aper, -y_aper, -y_aper]),
									   'x_sem_ellip_insc':0.99*x_aper,
									   'y_sem_ellip_insc':0.99*y_aper})
									   
    
poissonsolver = FD.FiniteDifferences_ShortleyWeller_SquareGrid(chamb=chamber, Dh=Dh, ext_boundary=True)


mesh = RectMesh2D(poissonsolver.bias_x,
                  poissonsolver.bias_y,
                  Dh, Dh,
                  poissonsolver.Nxg,
                  poissonsolver.Nyg)

poissonsolver_FFT = FFT.FFT_OpenBoundary_SquareGrid(x_aper=-poissonsolver.bias_x, y_aper=-poissonsolver.bias_y, Dh=Dh)

new_pp = PyPIC_Fortran_M2P_P2M(mesh, poissonsolver)#, gradient=poissonsolver.gradient)
new_pp_FFT = PyPIC_Fortran_M2P_P2M(mesh, poissonsolver_FFT)
picFDSW = PIC_FDSW.FiniteDifferences_ShortleyWeller_SquareGrid(chamb = chamber, Dh = Dh)
picFFTPEC = PIC_PEC_FFT.FFT_PEC_Boundary_SquareGrid(x_aper = chamber.x_aper, y_aper = chamber.y_aper, Dh = Dh, fftlib='pyfftw')
picFFT = PIC_FFT.FFT_OpenBoundary_SquareGrid(x_aper = chamber.x_aper, y_aper = chamber.y_aper, Dh = Dh, fftlib='pyfftw')

Nxg 129
Nyg 147
Mat. assembly 0%
Mat. assembly 5%
Mat. assembly 10%
Mat. assembly 15%
Mat. assembly 20%
Mat. assembly 25%
Mat. assembly 30%
Mat. assembly 35%
Mat. assembly 40%
Mat. assembly 45%
Mat. assembly 50%
Mat. assembly 55%
Mat. assembly 60%
Mat. assembly 65%
Mat. assembly 70%
Mat. assembly 75%
Mat. assembly 80%
Mat. assembly 85%
Mat. assembly 90%
Mat. assembly 95%
Mat. assembly 100%
Using scipy superlu solver...
Done PIC init.
129 147 0.1
Using PyFFTW
Start PIC init.:
Finite Differences, Shortley-Weller, Square Grid
Using Shortley-Weller boundary approx.
PyPIC Version 1.02
Mat. assembly 0%
Mat. assembly 5%
Mat. assembly 10%
Mat. assembly 15%
Mat. assembly 20%
Mat. assembly 25%
Mat. assembly 30%
Mat. assembly 35%
Mat. assembly 40%
Mat. assembly 45%
Mat. assembly 50%
Mat. assembly 55%
Mat. assembly 60%
Mat. assembly 65%
Mat. assembly 70%
Mat. assembly 75%
Mat. assembly 80%
Mat. assembly 85%
Mat. assembly 90%
Mat. assembly 95%
Mat. assembly 100%
Internal nodes with 0 potential
[]
U

../geom_impact_poly.py:157: RuntimeWarning: divide by zero encountered in divide
  (self.Nx[ii]*(x_out-x_in)+self.Ny[ii]*(y_out-y_in))


In [3]:
picFDSW.scatter(x_part, y_part, nel_part)
picFFTPEC.scatter(x_part, y_part, nel_part)
picFFT.scatter(x_part, y_part, nel_part)
mesh_charges = new_pp.particles_to_mesh(x_part, y_part, charge=e)


N_rep = 100

%timeit picFDSW.solve()

%timeit picFFTPEC.solve()

%timeit picFFT.solve()

%timeit new_pp.poisson_solve(mesh_charges)

%timeit new_pp_FFT.poisson_solve(mesh_charges)
# import time
# t_start_sw = time.mktime(time.localtime())
# for _ in range(N_rep):
# 	picFDSW.solve()
# t_stop_sw = time.mktime(time.localtime())
# t_sw = t_stop_sw-t_start_sw
# print 't_sw', t_sw


# t_start_fftpec = time.mktime(time.localtime())
# for _ in range(N_rep):
# 	picFFTPEC.solve()
# t_stop_fftpec = time.mktime(time.localtime())
# t_fftpec = t_stop_fftpec-t_start_fftpec
# print 't_fftpec', t_fftpec


# t_start_fftopen = time.mktime(time.localtime())
# for _ in range(N_rep):
# 	picFFT.solve()
# t_stop_fftopen = time.mktime(time.localtime())
# t_fftopen = t_stop_fftopen-t_start_fftopen
# print 't_fftopen', t_fftopen

The slowest run took 11.59 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 9.92 ms per loop
100 loops, best of 3: 4.4 ms per loop
100 loops, best of 3: 5.91 ms per loop
100 loops, best of 3: 14.3 ms per loop
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147 129
rho.shape (147, 129)
Nyg, Nxg open fft 147